<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-2-public/blob/adding_C4/C4/W2/ungraded_labs/C4_W2_Lab_2_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

##### Copyright 2019 The TensorFlow Authors.

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Classification

In this notebook we will classify movie reviews as being either `positive` or `negative`. We'll use the [IMDB dataset](https://www.tensorflow.org/datasets/catalog/imdb_reviews) that contains the text of 50,000 movie reviews from the [Internet Movie Database](https://www.imdb.com/). These are split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are *balanced*, meaning they contain an equal number of positive and negative reviews.

# Setup

In [3]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

print("\u2022 Using TensorFlow Version:", tf.__version__)

• Using TensorFlow Version: 2.15.0


## Download the IMDB Dataset

We will download the [IMDB dataset](https://www.tensorflow.org/datasets/catalog/imdb_reviews) using TensorFlow Datasets. We will use a training set, a validation set, and a test set. Since the IMDB dataset doesn't have a validation split, we will use the first 60\% of the training set for training, and the last 40\% of the training set for validation.

In [5]:
splits = ['train[:60%]', 'train[-40%:]', 'test']

splits, info = tfds.load(name="imdb_reviews", with_info=True, split=splits, as_supervised=True)

train_data, validation_data, test_data = splits

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Explore the Data

Let's take a moment to look at the data.

In [6]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples
num_classes = info.features['label'].num_classes

print('The Dataset has a total of:')
print('\u2022 {:,} classes'.format(num_classes))

print('\u2022 {:,} movie reviews for training'.format(num_train_examples))
print('\u2022 {:,} movie reviews for testing'.format(num_test_examples))

The Dataset has a total of:
• 2 classes
• 25,000 movie reviews for training
• 25,000 movie reviews for testing


The labels are either 0 or 1, where 0 is a negative review, and 1 is a positive review. We will create a list with the corresponding class names, so that we can map labels to class names later on.

In [7]:
class_names = ['negative', 'positive']

Each example consists of a sentence representing the movie review and a corresponding label. The sentence is not preprocessed in any way. Let's take a look at the first example of the training set.  

In [8]:
for review, label in train_data.take(1):
    review = review.numpy()
    label = label.numpy()

    print('\nMovie Review:\n\n', review)
    print('\nLabel:', class_names[label])


Movie Review:

 b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

Label: negative


## Load Word Embeddings

In this example, the input data consists of sentences. The labels to predict are either 0 or 1.

One way to represent the text is to convert sentences into word embeddings. Word embeddings, are an efficient way to represent words using dense vectors, where semantically similar words have similar vectors. We can use a pre-trained text embedding as the first layer of our model, which will have two advantages:

*   We don't have to worry anout text preprocessing.
*   We can benefit from transfer learning.

For this example we will use a model from [TensorFlow Hub](https://tfhub.dev/) called [google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1). We'll create a `hub.KerasLayer` that uses the TensorFlow Hub model to embed the sentences. We can choose to fine-tune the TF hub module weights during training by setting the `trainable` parameter to `True`.

In [9]:
# if you are running the notebook on Colab
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

# # if you are running the notebook on your local machine
# embedding = "./models/tf2-preview_gnews-swivel-20dim_1"

hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

## Build Pipeline

In [10]:
batch_size = 512

train_batches = train_data.shuffle(num_train_examples // 4).batch(batch_size).prefetch(1)
validation_batches = validation_data.batch(batch_size).prefetch(1)
test_batches = test_data.batch(batch_size)

## Build the Model

In the code below we will build a Keras `Sequential` model with the following layers:

1. The first layer is a TensorFlow Hub layer. This layer uses a pre-trained SavedModel to map a sentence into its embedding vector. The model that we are using ([google/tf2-preview/gnews-swivel-20dim/1](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1)) splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: `(num_examples, embedding_dimension)`.


2. This fixed-length output vector is piped through a fully-connected (`Dense`) layer with 16 hidden units.


3. The last layer is densely connected with a single output node. Using the `sigmoid` activation function, this value is a float between 0 and 1, representing a probability, or confidence level.

In [11]:
model = tf.keras.Sequential([
        hub_layer,
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')])

## Train the Model

Since this is a binary classification problem and the model outputs a probability (a single-unit layer with a sigmoid activation), we'll use the `binary_crossentropy` loss function.

In [12]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_batches,
                    epochs=20,
                    validation_data=validation_batches)

Epoch 1/20
30/30 [==============================] - 6s 148ms/step - loss: 0.7230 - accuracy: 0.5711 - val_loss: 0.6482 - val_accuracy: 0.6313
Epoch 2/20
30/30 [==============================] - 3s 94ms/step - loss: 0.6112 - accuracy: 0.6683 - val_loss: 0.5755 - val_accuracy: 0.6959
Epoch 3/20
30/30 [==============================] - 3s 94ms/step - loss: 0.5516 - accuracy: 0.7229 - val_loss: 0.5321 - val_accuracy: 0.7405
Epoch 4/20
30/30 [==============================] - 4s 129ms/step - loss: 0.5105 - accuracy: 0.7559 - val_loss: 0.4987 - val_accuracy: 0.7675
Epoch 5/20
30/30 [==============================] - 3s 93ms/step - loss: 0.4729 - accuracy: 0.7847 - val_loss: 0.4722 - val_accuracy: 0.7874
Epoch 6/20
30/30 [==============================] - 2s 75ms/step - loss: 0.4395 - accuracy: 0.8044 - val_loss: 0.4437 - val_accuracy: 0.8061
Epoch 7/20
30/30 [==============================] - 2s 77ms/step - loss: 0.4054 - accuracy: 0.8295 - val_loss: 0.4185 - val_accuracy: 0.8180
Epoch 8/20


## Evaluate the Model

We will now see how well our model performs on the testing set.

In [13]:
eval_results = model.evaluate(test_batches, verbose=0)

for metric, value in zip(model.metrics_names, eval_results):
    print(metric + ': {:.3}'.format(value))

loss: 0.324
accuracy: 0.864
